In [77]:
import plotly.express as ex
import sklearn as sk
import pandas as pd
import numpy as np
import random


In [78]:
actions_data = pd.read_csv("../data/actions.csv").dropna()
cars_data = pd.read_csv("../data/cars_about.csv").dropna()

In [79]:
cars_data

,car_id,car_model,used_label,price,exteriorColor,interiorColor,drivetrain,minMPG,maxMPG,fuelType,transmission,engine,mileage
0,1.0,Toyota Sienna SE,Used,$39,Red,Black,Front-wheel Drive,19.0,27.0,Gasoline,8-Speed Automatic,3.5L V6 24V PDI DOHC,29403.0
1,2.0,Ford F-150 Lariat,Used,$49,Shadow Black,Black,Four-wheel Drive,19.0,24.0,Gasoline,10-Speed Automatic,3.5L V6 24V PDI DOHC Twin Turbo,32929.0
2,3.0,RAM 1500 Laramie,Used,$41,Granite Crystal Clearcoat Metallic,Black,Four-wheel Drive,15.0,21.0,Gasoline,8-Speed Automatic,5.7L V8 16V MPFI OHV,23173.0
3,4.0,Honda Accord Sport SE,Used,$28,Gray,–,Front-wheel Drive,29.0,35.0,Gasoline,Automatic CVT,1.5L I4 16V GDI DOHC Turbo,10598.0
4,5.0,Lexus RX 350,Used,$49,Eminent White Pearl,Birch,Front-wheel Drive,20.0,27.0,Gasoline,8-Speed Automatic,3.5L V6 24V PDI DOHC,28137.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,401.0,Jeep Grand Cherokee Limited,Used,$33,Diamond Black,Black,Four-wheel Drive,18.0,25.0,Gasoline,8-Speed Automatic,3.6L V6 24V MPFI DOHC,43807.0
400,402.0,Toyota RAV4 Hybrid XLE,Toyota Certified,$30,Super White,Nutmeg,All-wheel Drive,34.0,30.0,Hybrid,Automatic CVT,2.5L I4 16V MPFI DOHC Hybrid,41944.0
401,403.0,Cadillac XT4 Sport,Used,$35,Black,Black,All-wheel Drive,22.0,29.0,Gasoline,9-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,49386.0
402,404.0,Chevrolet Tahoe LTZ,Used,$35,Tungsten Metallic,Jet Black,Four-wheel Drive,16.0,22.0,Gasoline,6-Speed Automatic,5.3L V8 16V GDI OHV,115795.0


In [80]:
actions_data

,car_id,user_id
0,84,0
1,171,0
2,80,0
3,126,0
4,184,0
...,...,...
2869,13,74
2870,17,74
2871,17,74
2872,16,74


In [81]:
actions_data = actions_data.astype(int)

In [124]:
user_data = pd.DataFrame(
    columns = sorted(actions_data.car_id.unique(), key=int), 
    index=actions_data.user_id.unique()
    )
user_data

,0,1,2,3,4,5,6,7,8,9,...,268,279,284,293,298,305,307,335,336,359
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


У меня есть действия пользователей в которые входит только информация о том лайкнул он авто или нет, мне кажется, что это довольно скупая информация. Для того чтобы диверсифицировать предпочтения сгенерированных пользователей я буду рандомно генерировать рейтинги для фильмов <br> Каким образом? <br> авто было добавлено в любимые? => random([3, 4, 5]) <br> С остальными авто поступаю иначе, генерирую для пользователя рандомное кол-во авто с низким рейтингом и уже эти авто раздаю рейтинги от 1-2

In [125]:
for user_id in actions_data.user_id:
    current_user_actions = actions_data.query(f"user_id == {user_id}")
    for car_id in current_user_actions.car_id:
        user_data[int(car_id)].iloc[user_id] = random.randint(4, 5) 

user_data

,0,1,2,3,4,5,6,7,8,9,...,268,279,284,293,298,305,307,335,336,359
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4,NaN,NaN,NaN,4,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,NaN,4,5,4,4,5,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,5,NaN,4,5,4,5,4,5,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
for row_idx in user_data.index:
    # specify the number of NaN values to replace with random numbers
    n = random.randint(0, 20)
    # get the columns with NaN values in the specified row
    cols_with_nans =  user_data.iloc[row_idx][ user_data.iloc[row_idx].isnull()].index.tolist()

    # randomly fill 'n' of the NaN values in the specified row with values between 0 and 10
    random_values = np.random.randint(1, 3, n)
    for col in np.random.choice(cols_with_nans, n, replace=False):
        user_data.loc[row_idx, col] = random_values[n - 1]

In [127]:
user_data

,0,1,2,3,4,5,6,7,8,9,...,268,279,284,293,298,305,307,335,336,359
0,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
1,NaN,1,NaN,NaN,1,NaN,1,NaN,NaN,1,...,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4,NaN,NaN,NaN,4,NaN,NaN
3,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,...,NaN,4,NaN,5,NaN,NaN,1,NaN,NaN,NaN
4,4,5,NaN,4,5,4,4,5,5,NaN,...,NaN,NaN,NaN,NaN,1,NaN,1,NaN,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,5,NaN,4,5,4,5,4,5,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,2,NaN,2,2,NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,2,NaN,NaN,NaN,2,NaN,2
72,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NaN,1,NaN,NaN,NaN,1,1,NaN,NaN,NaN,...,NaN,4,NaN,4,1,1,NaN,NaN,NaN,NaN


In [128]:
user_data.to_csv("../data/user_data.csv", index=False)